In [1]:
import pandas as pd
import requests
import os
import json
import numpy as np
import time

In [2]:
pd.set_option('max_colwidth',400)

In [3]:
df = pd.read_csv('mobike_shanghai_sample_updated.csv')

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 10 columns):
orderid             102361 non-null int64
bikeid              102361 non-null int64
userid              102361 non-null int64
start_time          102361 non-null object
start_location_x    102361 non-null float64
start_location_y    102361 non-null float64
end_time            102361 non-null object
end_location_x      102361 non-null float64
end_location_y      102361 non-null float64
track               102361 non-null object
dtypes: float64(4), int64(3), object(3)
memory usage: 7.8+ MB


In [65]:
df.head()

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track
0,78387,158357,10080,2016-08-20 06:57,121.348,31.389,2016-08-20 07:04,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#121.350,31.390#121.351,31.390#121.351,31.391#121.352,31.391#121.353,31.391#121.353,31.394#121.354,31.391#121.355,31.391#121.356,31.389#121.356,31.390#121.356,31.391#121.357,31.388#121.357,31.389"
1,891333,92776,6605,2016-08-29 19:09,121.508,31.279,2016-08-29 19:31,121.489,31.271,"121.489,31.270#121.489,31.271#121.490,31.270#121.490,31.271#121.491,31.271#121.492,31.270#121.492,31.271#121.493,31.269#121.493,31.270#121.494,31.268#121.494,31.269#121.495,31.269#121.495,31.270#121.496,31.270#121.497,31.270#121.497,31.271#121.498,31.272#121.499,31.272#121.499,31.273#121.500,31.273#121.500,31.274#121.501,31.274#121.502,31.273#121.502,31.274#121.503,31.273#121.504,31.273#121.50..."
2,1106623,152045,8876,2016-08-13 16:17,121.383,31.254,2016-08-13 16:36,121.405,31.248,"121.381,31.251#121.382,31.251#121.382,31.252#121.382,31.253#121.382,31.254#121.383,31.250#121.383,31.251#121.383,31.254#121.384,31.250#121.385,31.250#121.386,31.250#121.387,31.250#121.387,31.251#121.388,31.251#121.388,31.252#121.389,31.251#121.389,31.252#121.390,31.251#121.391,31.251#121.392,31.251#121.393,31.248#121.393,31.250#121.394,31.248#121.394,31.249#121.394,31.250#121.395,31.248#121.39..."
3,1389484,196259,10648,2016-08-23 21:34,121.484,31.320,2016-08-23 21:43,121.471,31.325,"121.471,31.325#121.472,31.325#121.473,31.324#121.473,31.325#121.474,31.322#121.474,31.323#121.475,31.320#121.475,31.321#121.475,31.322#121.476,31.320#121.477,31.320#121.478,31.320#121.479,31.320#121.480,31.320#121.481,31.320#121.482,31.320#121.483,31.320#121.484,31.320"
4,188537,78208,11735,2016-08-16 07:32,121.407,31.292,2016-08-16 07:41,121.418,31.288,"121.407,31.291#121.407,31.292#121.408,31.291#121.411,31.288#121.414,31.288#121.417,31.288#121.418,31.287#121.418,31.288"


In [62]:
df.orderid.nunique()

102361

In [30]:
df_t = pd.DataFrame()

In [31]:
df_t['state'] = df.groupby(['userid']).count()['orderid']

In [32]:
df = pd.merge(df, df_t, on = ['userid'], how = 'inner')

In [33]:
df.head()

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track,state
0,78387,158357,10080,2016-08-20 06:57,121.348,31.389,2016-08-20 07:04,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#121.350,31.390#121.351,31.390#121.351,31.391#121.352,31.391#121.353,31.391#121.353,31.394#121.354,31.391#121.355,31.391#121.356,31.389#121.356,31.390#121.356,31.391#121.357,31.388#121.357,31.389",6
1,256597,138691,10080,2016-08-17 07:08,121.437,31.330,2016-08-17 07:13,121.440,31.331,"121.437,31.329#121.437,31.330#121.438,31.330#121.439,31.330#121.440,31.330#121.440,31.331",6
2,448255,94634,10080,2016-08-05 16:59,121.480,31.226,2016-08-05 17:13,121.463,31.225,"121.463,31.224#121.463,31.225#121.464,31.224#121.465,31.224#121.466,31.224#121.467,31.223#121.467,31.224#121.468,31.223#121.469,31.223#121.470,31.223#121.471,31.223#121.472,31.223#121.473,31.223#121.473,31.224#121.474,31.224#121.475,31.224#121.476,31.224#121.476,31.225#121.477,31.225#121.478,31.226#121.479,31.226#121.480,31.226",6
3,283591,1663,10080,2016-08-05 14:35,121.478,31.215,2016-08-05 14:41,121.477,31.221,"121.477,31.220#121.477,31.221#121.478,31.215#121.478,31.216#121.478,31.217#121.478,31.218#121.478,31.219#121.478,31.220",6
4,1260579,171434,10080,2016-08-12 17:54,121.482,31.281,2016-08-12 18:02,121.475,31.288,"121.475,31.288#121.476,31.288#121.477,31.286#121.477,31.287#121.477,31.288#121.478,31.286#121.479,31.286#121.480,31.285#121.480,31.286#121.481,31.282#121.481,31.283#121.481,31.284#121.481,31.285#121.482,31.282",6


In [10]:
df.user_level.value_counts()

老客户     76766
粘性客户    24401
新客户      1194
Name: user_level, dtype: int64

In [5]:
def add_user_state(df):
    start_time = time.time()
    df = df.apply(lambda x: user_state(x), axis = 1)
    end_time = time.time()
    print("add_user_state()用时：" + str((end_time - start_time)/60) + "分钟")
    return df

In [6]:
def user_state(data):
    if data['state'] == 1:
        data['user_level'] = '新客户'
    if data['state'] <= 10 and data['state'] > 1:
        data['user_level'] = '老客户'
    if data['state'] > 10:
        data['user_level'] = '粘性客户'
    return data

In [7]:
df = add_user_state(df)

add_user_state()用时：0.23087348540623984分钟


In [13]:
df.head()

,orderid,bikeid,userid,start_time,start_location_x,start_location_y,end_time,end_location_x,end_location_y,track,state,user_level
0,78387,158357,10080,2016-08-20 06:57,121.348,31.389,2016-08-20 07:04,121.357,31.388,"121.347,31.392#121.348,31.389#121.349,31.390#121.350,31.390#121.351,31.390#121.351,31.391#121.352,31.391#121.353,31.391#121.353,31.394#121.354,31.391#121.355,31.391#121.356,31.389#121.356,31.390#121.356,31.391#121.357,31.388#121.357,31.389",6,老客户
1,256597,138691,10080,2016-08-17 07:08,121.437,31.330,2016-08-17 07:13,121.440,31.331,"121.437,31.329#121.437,31.330#121.438,31.330#121.439,31.330#121.440,31.330#121.440,31.331",6,老客户
2,448255,94634,10080,2016-08-05 16:59,121.480,31.226,2016-08-05 17:13,121.463,31.225,"121.463,31.224#121.463,31.225#121.464,31.224#121.465,31.224#121.466,31.224#121.467,31.223#121.467,31.224#121.468,31.223#121.469,31.223#121.470,31.223#121.471,31.223#121.472,31.223#121.473,31.223#121.473,31.224#121.474,31.224#121.475,31.224#121.476,31.224#121.476,31.225#121.477,31.225#121.478,31.226#121.479,31.226#121.480,31.226",6,老客户
3,283591,1663,10080,2016-08-05 14:35,121.478,31.215,2016-08-05 14:41,121.477,31.221,"121.477,31.220#121.477,31.221#121.478,31.215#121.478,31.216#121.478,31.217#121.478,31.218#121.478,31.219#121.478,31.220",6,老客户
4,1260579,171434,10080,2016-08-12 17:54,121.482,31.281,2016-08-12 18:02,121.475,31.288,"121.475,31.288#121.476,31.288#121.477,31.286#121.477,31.287#121.477,31.288#121.478,31.286#121.479,31.286#121.480,31.285#121.480,31.286#121.481,31.282#121.481,31.283#121.481,31.284#121.481,31.285#121.482,31.282",6,老客户


In [12]:
df.to_csv('mobike_shanghai_sample_updated.csv', index=False)

In [4]:
df_l = df[['orderid', 'start_location_x', 'start_location_y', 'end_location_x', 'end_location_y']]

In [5]:
df_l.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102361 entries, 0 to 102360
Data columns (total 5 columns):
orderid             102361 non-null int64
start_location_x    102361 non-null float64
start_location_y    102361 non-null float64
end_location_x      102361 non-null float64
end_location_y      102361 non-null float64
dtypes: float64(4), int64(1)
memory usage: 3.9 MB


In [6]:
import geopy.geocoders
from geopy.geocoders import Baidu
geopy.geocoders.options.default_user_agent = 'mobike'
geopy.geocoders.options.default_timeout = 60000

In [7]:
geolocator = Baidu(api_key='PvP87YrUMvAKmbR2FeSOExP47MViQKlr')

测试

In [9]:
a = geolocator.reverse((31.389, 121.348)).address.decode('utf-8')
print(a)

上海市宝山区美文路


In [24]:
def add_location(df):
    start_time = time.time()
    df['start_loction'] = df.apply(lambda x: geolocator.reverse(query=(x['start_location_y'], x['start_location_x'])).address.decode('utf-8'), axis = 1)
    df['end_loction'] = df.apply(lambda x: geolocator.reverse(query=(x['end_location_y'], x['end_location_x'])).address.decode('utf-8'), axis = 1)
    end_time = time.time()
    print("add_location()用时：" + str((end_time - start_time)/60) + "分钟")
    return df

In [21]:
df_1000 = df_l.iloc[0:1000, :]

In [22]:
df_1000 = add_location(df_1000)

/Users/xiong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


add_location()用时：494.19729685783386


/Users/xiong/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [23]:
df_1000.head(10)

,orderid,start_location_x,start_location_y,end_location_x,end_location_y,start_loction,end_loction
0,78387,121.348,31.389,121.357,31.388,上海市宝山区美文路,上海市宝山区长联路357号-临
1,891333,121.508,31.279,121.489,31.271,上海市虹口区阜新路180弄1号西,上海市虹口区四川北路2084号
2,1106623,121.383,31.254,121.405,31.248,上海市嘉定区梅川路1558弄-39号,上海市普陀区梅岭北路992弄18号
3,1389484,121.484,31.320,121.471,31.325,上海市虹口区安汾路588号,上海市闸北区阳泉路693号
4,188537,121.407,31.292,121.418,31.288,上海市普陀区真大路410号,上海市宝山区真华路1376号
5,537030,121.488,31.224,121.468,31.233,上海市黄浦区复兴中路71号,上海市静安区威海路523号
6,517706,121.445,31.176,121.447,31.192,上海市徐汇区漕东支路218弄1号501室,上海市徐汇区零陵路777弄77
7,270836,121.369,31.252,121.376,31.233,上海市嘉定区丰庄路303号,上海市普陀区云岭西路
8,441362,121.529,31.324,121.532,31.323,上海市杨浦区国和路858号,上海市杨浦区世界路51号
9,76435,121.467,31.320,121.447,31.318,上海市闸北区阳曲路470弄-1号,上海市闸北区三泉路1495号


In [22]:
def json_to_df(json_url):
    response = requests.get(json_url)
    answer = response.json()
    df_json = pd.DataFrame(answer['features'])
    return df_json

将json文件读取到DataFrame df_json，进行了数据清洗

In [21]:
df_json = json_to_df('https://geo.datav.aliyun.com/areas/bound/310000_full.json')

TypeError: list indices must be integers or slices, not str

检验

In [15]:
df_json.head()

,geometry,properties,type
0,"{'type': 'MultiPolygon', 'coordinates': [[[[12...","{'adcode': 310101, 'name': '黄浦区', 'center': [1...",Feature
1,"{'type': 'MultiPolygon', 'coordinates': [[[[12...","{'adcode': 310104, 'name': '徐汇区', 'center': [1...",Feature
2,"{'type': 'MultiPolygon', 'coordinates': [[[[12...","{'adcode': 310105, 'name': '长宁区', 'center': [1...",Feature
3,"{'type': 'MultiPolygon', 'coordinates': [[[[12...","{'adcode': 310106, 'name': '静安区', 'center': [1...",Feature
4,"{'type': 'MultiPolygon', 'coordinates': [[[[12...","{'adcode': 310107, 'name': '普陀区', 'center': [1...",Feature


In [23]:
df_json.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 3 columns):
geometry      16 non-null object
properties    16 non-null object
type          16 non-null object
dtypes: object(3)
memory usage: 464.0+ bytes


In [25]:
url = 'https://geo.datav.aliyun.com/areas/bound/310000_full.json'
reponse = requests.get(url)
with open(url.split('/')[-1], mode='wb') as file:
    file.write(reponse.content)

方法2：
1. 通过在http://datav.aliyun.com/tools/atlas/#&lat=45.10454630976873&lng=106.94091796875&zoom=5 下载对应的城市json数据，该文件格式可以被tableau识别为geojson格式
2. 使用geojson_utils.py工具，下载和使用方法：https://github.com/brandonxiang/geojson-python-utils/blob/develop/README_CN.md
3. 处理原始数据集，通过`add_adcode`增加区号内容列adcode，目的是作为唯一值，在tableau与geojson文件联合。

In [30]:
from geojson_utils import point_in_multipolygon

In [24]:
in_str = '{"type": "Point", "coordinates": [121.459, 31.229]}'
box_str = '{"type": "Polygon","coordinates": [[[121.49483,31.24221],[121.49349,31.24017],[121.49349,31.23615],[121.50672,31.22314],[121.50984,31.21674],[121.50925,31.21211],[121.50132,31.19974],[121.49597,31.19381],[121.49324,31.19216],[121.47598,31.18789],[121.47446,31.19022],[121.47036,31.19122],[121.46687,31.20426],[121.46227,31.20317],[121.46123,31.21172],[121.45676,31.2239],[121.46749,31.22387],[121.46632,31.23431],[121.46297,31.2414],[121.46956,31.23921],[121.4756,31.24149],[121.48058,31.24017],[121.48697,31.24424],[121.49483,31.24221]]]}'
in_box = json.loads(in_str)
box = json.loads(box_str)
print(point_in_polygon(in_box, box))

False


In [31]:
with open("310000_full.json",'r') as load_f:
     load_dict = json.load(load_f)
     print(load_dict['features'][0]['geometry'])

{'type': 'MultiPolygon', 'coordinates': [[[[121.475403, 31.188004], [121.474777, 31.190233], [121.470328, 31.191252], [121.469563, 31.196512], [121.46663, 31.204347], [121.46225, 31.20297], [121.461555, 31.210194], [121.460707, 31.213488], [121.457689, 31.220184], [121.456758, 31.223898], [121.467492, 31.223874], [121.466226, 31.234641], [121.462987, 31.241372], [121.468534, 31.23948], [121.470773, 31.239444], [121.475764, 31.241492], [121.479977, 31.240115], [121.483008, 31.241995], [121.485761, 31.244019], [121.487777, 31.244198], [121.494909, 31.242366], [121.493102, 31.238282], [121.494381, 31.234605], [121.499998, 31.229994], [121.506477, 31.223622], [121.509605, 31.217908], [121.510036, 31.214099], [121.50524, 31.205486], [121.500638, 31.198608], [121.496008, 31.193648], [121.489529, 31.191072], [121.475403, 31.188004]]]]}


In [32]:
def add_adcode(df):
    start_time = time.time()
    df = df.apply(lambda x: is_inarea(x), axis = 1)
    end_time = time.time()
    print("add_adcode()用时：" + str((end_time - start_time)/60) + "分钟")
    return df

In [33]:
def is_inarea(data):
    for i in range(len(load_dict['features'])):
        loc_str ='{"type": "Point", "coordinates": [' +  str(data.start_location_x) + ', ' + str(data.start_location_y) + ']}'
        box_str = json.dumps(load_dict['features'][i]['geometry'])
        in_box = json.loads(loc_str)
        box = json.loads(box_str)
        if point_in_multipolygon(in_box, box):
            data['adcode'] = load_dict['features'][i]['properties']['adcode']
            data['dis_name'] = load_dict['features'][i]['properties']['name']
    return data

In [34]:
df = add_adcode(df)

add_adcode()用时：16.185615118344625分钟


In [35]:
df[df['adcode'].isnull().values==True]

,adcode,bikeid,dis_name,end_location_x,end_location_y,end_time,orderid,start_location_x,start_location_y,start_time,state,track,user_level,userid


In [37]:
df.head()

,adcode,bikeid,dis_name,end_location_x,end_location_y,end_time,orderid,start_location_x,start_location_y,start_time,state,track,user_level,userid
0,310113,158357,宝山区,121.357,31.388,2016-08-20 07:04,78387,121.348,31.389,2016-08-20 06:57,6,"121.347,31.392#121.348,31.389#121.349,31.390#121.350,31.390#121.351,31.390#121.351,31.391#121.352,31.391#121.353,31.391#121.353,31.394#121.354,31.391#121.355,31.391#121.356,31.389#121.356,31.390#121.356,31.391#121.357,31.388#121.357,31.389",老客户,10080
1,310113,138691,宝山区,121.440,31.331,2016-08-17 07:13,256597,121.437,31.330,2016-08-17 07:08,6,"121.437,31.329#121.437,31.330#121.438,31.330#121.439,31.330#121.440,31.330#121.440,31.331",老客户,10080
2,310101,94634,黄浦区,121.463,31.225,2016-08-05 17:13,448255,121.480,31.226,2016-08-05 16:59,6,"121.463,31.224#121.463,31.225#121.464,31.224#121.465,31.224#121.466,31.224#121.467,31.223#121.467,31.224#121.468,31.223#121.469,31.223#121.470,31.223#121.471,31.223#121.472,31.223#121.473,31.223#121.473,31.224#121.474,31.224#121.475,31.224#121.476,31.224#121.476,31.225#121.477,31.225#121.478,31.226#121.479,31.226#121.480,31.226",老客户,10080
3,310101,1663,黄浦区,121.477,31.221,2016-08-05 14:41,283591,121.478,31.215,2016-08-05 14:35,6,"121.477,31.220#121.477,31.221#121.478,31.215#121.478,31.216#121.478,31.217#121.478,31.218#121.478,31.219#121.478,31.220",老客户,10080
4,310109,171434,虹口区,121.475,31.288,2016-08-12 18:02,1260579,121.482,31.281,2016-08-12 17:54,6,"121.475,31.288#121.476,31.288#121.477,31.286#121.477,31.287#121.477,31.288#121.478,31.286#121.479,31.286#121.480,31.285#121.480,31.286#121.481,31.282#121.481,31.283#121.481,31.284#121.481,31.285#121.482,31.282",老客户,10080


In [36]:
df.to_csv('mobike_shanghai_sample_updated.csv', index=False)